In [1]:
from sklearn.datasets import load_iris
import pandas as pd

#Cargar el conjunto de datos Iris
iris = load_iris()
iris_df = pd.DataFrame(data=iris['data'],columns=iris['feature_names'])
iris_df['target'] = iris['target']

#Mostrar las primeras filas del conjunto de datos
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
#Super Vector Machine
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

#Dividir el conjunto de datos en entrenamiento y prueba
x = iris_df.drop('target',axis=1)
y = iris_df['target']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#Definir el modelo SVC
svm_model = SVC()

#Mostrar los hiperparámetros por defecto del modelo
svm_model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

GridSearch

In [18]:
from sklearn.model_selection import GridSearchCV

#Definir la cuadrícula de hiperparámetros
param_grid = {
   'C' : [0.1, 1, 10, 100],
   'kernel' : ['linear', 'rbf'],
   'gamma' : ['scale', 'auto']
}

#Crear el objeto GridSearchCV
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)

#Ajustar el modelo utilizando la búsqueda en grid
grid_search.fit(x_train, y_train)

#Obtener los mejores hiperparámetros encontrados
best_params_grid_search = grid_search.best_params_
best_params_grid_search

{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}

RandomSearch

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

#Definir la distribución de hiperparámetros
param_dist = {
   'C' : uniform(0.1,100),
   'kernel' : ['linear', 'rbf'],
   'gamma' : ['scale', 'auto']
}

#Crear el objeto RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_dist, n_iter=100, cv=5, random_state=42)

#Ajustar el modelo utilizando la búsqueda aleatoria
random_search.fit(x_train, y_train)

#Obtener los mejores hiperparámetros encontrados
best_params_random_search = random_search.best_params_
best_params_random_search

{'C': 3.5388521115218396, 'gamma': 'auto', 'kernel': 'linear'}

Optimización Bayesiana

In [23]:
from skopt import BayesSearchCV

#Definir un espacio de búsqueda reducido para la optimización bayesiana
param_space_reduced = {
   'C' : (0.1,5),
   'kernel' : ['linear', 'rbf'],
   'gamma' : ['scale', 'auto']
}

#Crear el objeto BayesSearchCV con un número reducido de iteraciones
bayes_search_reduced = BayesSearchCV(estimator=svm_model, search_spaces=param_space_reduced, n_iter=25, cv=5, random_state=42)

#Ajustar el modelo utilizando la optimización bayesiana con el espacio de búsqueda reducido
bayes_search_reduced.fit(x_train, y_train)

#Obtener los mejores hiperparámetros encontrados
best_params_bayes_search = bayes_search_reduced.best_params_
best_params_bayes_search

c:\Python311\Lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


OrderedDict([('C', 4.203202942211093),
             ('gamma', 'scale'),
             ('kernel', 'linear')])

Evaluación de Modelos con el cambio de los hiperparámetros

In [24]:
from sklearn.metrics import accuracy_score

#Evaluar el modelo obtenido con la búsqueda en Grid
grid_search_model = grid_search.best_estimator_
grid_search_predictions = grid_search_model.predict(x_test)
grid_search_accuracy = accuracy_score(y_test, grid_search_predictions)

#Evaluar el modelo obtenido con la búsqueda aleatoria
random_search_model = random_search.best_estimator_
random_search_predictions = random_search_model.predict(x_test)
random_search_accuracy = accuracy_score(y_test, random_search_predictions)

#Evaluar el modelo obtenido con la optimización bayesiana
bayes_search_model = bayes_search_reduced.best_estimator_
bayes_search_predictions = bayes_search_model.predict(x_test)
bayes_search_accuracy = accuracy_score(y_test, bayes_search_predictions)

grid_search_accuracy, random_search_accuracy, bayes_search_accuracy

(1.0, 0.9666666666666667, 0.9666666666666667)

In [21]:
from sklearn.model_selection import cross_val_score

#Utilizar el mejor modelo encontrado por la búsqueda Grid
best_model = grid_search.best_estimator_

#Realizar una validación cruzada de 5 pliegues en el conjunto de datos completo (x,y)
cv_scores = cross_val_score(best_model, x, y, cv=5)

#Calcular la media y la desviación estándar de los puntajes de validación cruzada
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

cv_mean, cv_std

(0.9800000000000001, 0.016329931618554516)

In [25]:
#Utilizar el mejor modelo encontrado por la búsqueda aleatoria
best_model_random_search = random_search.best_estimator_

#Utilizar el mejor modelo encontrado por la optimización bayesiana
best_model_bayes_search = bayes_search_reduced.best_estimator_

#Realizar una validación cruzada de 5 pliegues en el conjunto de datos completo (x,y)
cv_scores_grid_search = cross_val_score(best_model, x, y, cv=5)
cv_scores_random_search = cross_val_score(best_model_random_search, x, y, cv=5)
cv_scores_bayes_search = cross_val_score(best_model_bayes_search, x, y, cv=5)

In [26]:
#Calcular la media y la desviación estándar de los puntajes de validación cruzada
cv_mean_grid_search, cv_std_grid_search = cv_scores_grid_search.mean(), cv_scores_grid_search.std()
cv_mean_random_search, cv_std_random_search = cv_scores_random_search.mean(), cv_scores_random_search.std()
cv_mean_bayes_search, cv_std_bayes_search = cv_scores_bayes_search.mean(), cv_scores_bayes_search.std()

In [27]:
#Imprimir resultados
print(f'Bayes --> mean = {cv_mean_grid_search} ; std = {cv_std_grid_search}')
print(f'Random --> mean = {cv_mean_random_search} ; std = {cv_std_random_search}')
print(f'Bayes --> mean = {cv_mean_bayes_search} ; std = {cv_std_bayes_search}')

Bayes --> mean = 0.9800000000000001 ; std = 0.016329931618554516
Random --> mean = 0.9733333333333334 ; std = 0.02494438257849294
Bayes --> mean = 0.9733333333333334 ; std = 0.02494438257849294
